In [1]:
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

/home/fzuo/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Benign examples

In [2]:
with open('/imgNet/pca/rd_by_pca_ben_matrix_120D.pkl', 'rb') as handler:
    x_ben_imgs = pickle.load(handler)
print(x_ben_imgs.shape)

X_train = x_ben_imgs[:24000]
X_test = x_ben_imgs[24000:30000]
print(X_train.shape)
print(X_test.shape)

(30000, 120)
(24000, 120)
(6000, 120)


## Adversarial examples

In [3]:
#CW (gamma = 0.0, 0.4, 0.1)
with open('/imgNet/pca/cw00/rd_by_pca_adv_00_matrix_120D.pkl', 'rb') as handler:
    x_adv_imgs_00 = pickle.load(handler)

with open('/imgNet/pca/cw04/rd_by_pca_adv_04_matrix_120D.pkl', 'rb') as handler:
    x_adv_imgs_04 = pickle.load(handler)

with open('/imgNet/pca/cw10/rd_by_pca_adv_10_matrix_120D.pkl', 'rb') as handler:
    x_adv_imgs_10 = pickle.load(handler)
    
#DeepFool
with open('/imgNet/pca/dfool/rd_by_pca_adv_df_matrix_120D.pkl', 'rb') as handler:
    x_adv_imgs_df = pickle.load(handler)

## Data split

In [4]:
X_train = np.concatenate((X_train, x_adv_imgs_00[:4000]))
X_train = np.concatenate((X_train, x_adv_imgs_04[:4000]))
X_train = np.concatenate((X_train, x_adv_imgs_10[:4000]))
X_train = np.concatenate((X_train, x_adv_imgs_df[:12000]))

X_test = np.concatenate((X_test, x_adv_imgs_00[4000:5000]))
X_test = np.concatenate((X_test, x_adv_imgs_04[4000:5000]))
X_test = np.concatenate((X_test, x_adv_imgs_10[4000:5000]))
X_test = np.concatenate((X_test, x_adv_imgs_df[12000:15000]))

y_train = np.append(np.zeros(24000), np.ones(24000))
y_test = np.append(np.zeros(6000), np.ones(6000))

In [5]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(48000, 120)
(12000, 120)
(48000,)
(12000,)


## detector based on AdaBoost 

In [6]:
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)

bdt.fit(X_train, y_train)

print(bdt.score(X_train, y_train))
print(bdt.score(X_test, y_test))

0.9730625
0.9715


In [7]:
joblib.dump(bdt,'adaboost_imgNet_ResNet50_cw_df.pkl')
clf_bdt=joblib.load('adaboost_imgNet_ResNet50_cw_df.pkl')

In [8]:
print(clf_bdt.score(X_test[:6000], y_test[:6000]))               #97.3%
print(clf_bdt.score(X_test[6000:7000], y_test[6000:7000]))       #98.9%
print(clf_bdt.score(X_test[7000:8000], y_test[7000:8000]))       #99.2%
print(clf_bdt.score(X_test[8000:9000], y_test[8000:9000]))       #99.3%
print(clf_bdt.score(X_test[9000:12000], y_test[9000:12000]))     #95.0%

0.9725
0.989
0.992
0.993
0.9496666666666667


## detector based on SVM

In [9]:
svc_p = SVC(kernel = 'rbf', gamma = 0.01, C = 5)
svc_p.fit(X_train, y_train)

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
joblib.dump(svc_p,'svm_imgNet_ResNet50_cw_df.pkl')
clf_svc=joblib.load('svm_imgNet_ResNet50_cw_df.pkl')

In [11]:
print(clf_svc.score(X_test[:6000], y_test[:6000]))            #96.5%
print(clf_svc.score(X_test[6000:7000], y_test[6000:7000]))    #97.9%
print(clf_svc.score(X_test[7000:8000], y_test[7000:8000]))    #98.4%
print(clf_svc.score(X_test[8000:9000], y_test[8000:9000]))    #98.7%
print(clf_svc.score(X_test[9000:12000], y_test[9000:12000]))  #93.7%

0.9651666666666666
0.979
0.984
0.987
0.9373333333333334
